# Compare Monthly PET between Pathways

Two pathways are used within the framework.

1. H0: null hypothesis of historical weather statistics
2. H1: alternative hypothesis of weather statistics extracted from down-scaled, global climate model results.

There are four analysis periods

1. Data Period: 1981-2010
2. Projection Period 1: 2011-2040
3. Projection Period 2: 2041-2070
4. Projection Period 3: 2071-2100

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
from math import floor, ceil
from copy import deepcopy

In [3]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [4]:
# input directories and file roots
IN_DIR = r'C:\Temp\WG_Test_Out\Test2'
OUT_DIR = r'C:\Temp\WG_Test_Out\Test2\Processed'
OUT_ROOT = "DC_WGMN2"
IN_H0ROOT1 = "WB_H0_%s_R" % OUT_ROOT
IN_H1ROOT1 = "WB_H1_%s_R" % OUT_ROOT
IN_ROOT2 = "_DF.pickle"
NUMREAL = 10000

In [5]:
# time periods
TP_DICT = { 1 : [ dt.datetime(1981, 1, 1), dt.datetime(2010, 12, 31)],
            2 : [ dt.datetime(2011, 1, 1), dt.datetime(2040, 12, 31)],
            3 : [ dt.datetime(2041, 1, 1), dt.datetime(2070, 12, 31)],
            4 : [ dt.datetime(2071, 1, 1), dt.datetime(2100, 12, 31)],}

In [6]:
OurQs = np.array( [ round( 0.01 * x, 2 ) for x in range(101) ], dtype=np.float32 )
NumQs = len( OurQs )
NumQs

101

In [7]:
StatsHdsList = list()
for iI in range(NumQs):
    cQ = OurQs[iI]
    Hdr = "%d_ptile" % round(cQ * 100.0)
    StatsHdsList.append( Hdr )
# end of for
StatsHdsList.append( "Average" )
StatsHdsList.append( "Variance" )
StatsHdsList.append( "Skew" )
StatsHdsList.append( "Kurt" )

In [8]:
NumHdrs = len( StatsHdsList )
NumHdrs

105

In [9]:
WB_In_Hdrs = [ "Precip_mm", #0
               "ETo_mm", #1
               "PET_mm", #2
               "P-PET_mm", #3
               "APWL_mm", #4
               "SM_mm", #5
               "DelSM_mm", #6
               "AET_mm", #7
               "Def_mm", #8
               "Surp_mm", #9
               "TotAvail_mm", #10
               "RO_mm", #11
               "Detent_mm", #12
               "Re_mm", #13
             ]

## Collate DataFrames

### H0

In [10]:
for rR in range( 1, NUMREAL + 1, 1 ):
    if ((rR % 500) == 0):
        print("Working on realization %d" % rR)
    # end if
    InFile = "%s%d%s" % ( IN_H0ROOT1, rR, IN_ROOT2 )
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    if rR <= 1:
        # data period
        H0PET1 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0PET1.columns = ["R_%d" % rR]
        # Projection Period 1
        H0PET2 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0PET2.columns = ["R_%d" % rR]
        # Projection Period 2
        H0PET3 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0PET3.columns = ["R_%d" % rR]
        # Projection Period 3
        H0PET4 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0PET4.columns = ["R_%d" % rR]
    else:
        # data period
        tmpH0PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpH0PET.columns = ["R_%d" % rR]
        H0PET1 = H0PET1.merge( tmpH0PET, how='inner', left_index=True, right_index=True )
        # Projection Period 1
        tmpH0PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpH0PET.columns = ["R_%d" % rR]
        H0PET2 = H0PET2.merge( tmpH0PET, how='inner', left_index=True, right_index=True )
        # Projection Period 2
        tmpH0PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpH0PET.columns = ["R_%d" % rR]
        H0PET3 = H0PET3.merge( tmpH0PET, how='inner', left_index=True, right_index=True )
        # Projection Period 3
        tmpH0PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpH0PET.columns = ["R_%d" % rR]
        H0PET4 = H0PET4.merge( tmpH0PET, how='inner', left_index=True, right_index=True )
    # end of if
# end of for

Working on realization 500
Working on realization 1000
Working on realization 1500
Working on realization 2000
Working on realization 2500
Working on realization 3000
Working on realization 3500
Working on realization 4000
Working on realization 4500
Working on realization 5000
Working on realization 5500
Working on realization 6000
Working on realization 6500
Working on realization 7000
Working on realization 7500
Working on realization 8000
Working on realization 8500
Working on realization 9000
Working on realization 9500
Working on realization 10000


In [11]:
OFNamer = "WBMon_H0_PET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0PET1.to_pickle( OFPath, compression='zip' )
#H0PET1 = pd.read_pickle( OFPath, compression='zip' )
OFNamer = "WBMon_H0_PET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0PET2.to_pickle( OFPath, compression='zip' )
#H0PET2 = pd.read_pickle( OFPath, compression='zip' )
OFNamer = "WBMon_H0_PET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0PET3.to_pickle( OFPath, compression='zip' )
#H0PET3 = pd.read_pickle( OFPath, compression='zip' )
OFNamer = "WBMon_H0_PET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0PET4.to_pickle( OFPath, compression='zip' )
#H0PET4 = pd.read_pickle( OFPath, compression='zip' )

### H1

In [12]:
for rR in range( 1, NUMREAL + 1, 1 ):
    if ((rR % 500) == 0):
        print("Working on realization %d" % rR)
    # end if
    InFile = "%s%d%s" % ( IN_H1ROOT1, rR, IN_ROOT2 )
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    if rR <= 1:
        # data period
        H1PET1 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1PET1.columns = ["R_%d" % rR]
        # Projection Period 1
        H1PET2 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1PET2.columns = ["R_%d" % rR]
        # Projection Period 2
        H1PET3 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1PET3.columns = ["R_%d" % rR]
        # Projection Period 3
        H1PET4 = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1PET4.columns = ["R_%d" % rR]
    else:
        # data period
        tmpH1PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpH1PET.columns = ["R_%d" % rR]
        H1PET1 = H1PET1.merge( tmpH1PET, how='inner', left_index=True, right_index=True )
        # Projection Period 1
        tmpH1PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpH1PET.columns = ["R_%d" % rR]
        H1PET2 = H1PET2.merge( tmpH1PET, how='inner', left_index=True, right_index=True )
        # Projection Period 2
        tmpH1PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpH1PET.columns = ["R_%d" % rR]
        H1PET3 = H1PET3.merge( tmpH1PET, how='inner', left_index=True, right_index=True )
        # Projection Period 3
        tmpH1PET = InDF[[WB_In_Hdrs[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpH1PET.columns = ["R_%d" % rR]
        H1PET4 = H1PET4.merge( tmpH1PET, how='inner', left_index=True, right_index=True )
    # end of if
# end of for

Working on realization 500
Working on realization 1000
Working on realization 1500
Working on realization 2000
Working on realization 2500
Working on realization 3000
Working on realization 3500
Working on realization 4000
Working on realization 4500
Working on realization 5000
Working on realization 5500
Working on realization 6000
Working on realization 6500
Working on realization 7000
Working on realization 7500
Working on realization 8000
Working on realization 8500
Working on realization 9000
Working on realization 9500
Working on realization 10000


In [13]:
OFNamer = "WBMon_H1_PET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1PET1.to_pickle( OFPath, compression='zip' )
#H1PET1 = pd.read_pickle( OFPath, compression='zip' )
OFNamer = "WBMon_H1_PET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1PET2.to_pickle( OFPath, compression='zip' )
#H1PET2 = pd.read_pickle( OFPath, compression='zip' )
OFNamer = "WBMon_H1_PET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1PET3.to_pickle( OFPath, compression='zip' )
#H1PET3 = pd.read_pickle( OFPath, compression='zip' )
OFNamer = "WBMon_H1_PET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1PET4.to_pickle( OFPath, compression='zip' )
#H1PET4 = pd.read_pickle( OFPath, compression='zip' )

## Make PET Pivot Tables

In [14]:
H0PET1["Year"] = H0PET1.index.year
H0PET1["Month"] = H0PET1.index.month
H0PET2["Year"] = H0PET2.index.year
H0PET2["Month"] = H0PET2.index.month
H0PET3["Year"] = H0PET3.index.year
H0PET3["Month"] = H0PET3.index.month
H0PET4["Year"] = H0PET4.index.year
H0PET4["Month"] = H0PET4.index.month
H0PET1["Average"] = H0PET1.mean( axis=1 )
H0PET2["Average"] = H0PET2.mean( axis=1 )
H0PET3["Average"] = H0PET3.mean( axis=1 )
H0PET4["Average"] = H0PET4.mean( axis=1 )

In [15]:
H1PET1["Year"] = H1PET1.index.year
H1PET1["Month"] = H1PET1.index.month
H1PET2["Year"] = H1PET2.index.year
H1PET2["Month"] = H1PET2.index.month
H1PET3["Year"] = H1PET3.index.year
H1PET3["Month"] = H1PET3.index.month
H1PET4["Year"] = H1PET4.index.year
H1PET4["Month"] = H1PET4.index.month
H1PET1["Average"] = H1PET1.mean( axis=1 )
H1PET2["Average"] = H1PET2.mean( axis=1 )
H1PET3["Average"] = H1PET3.mean( axis=1 )
H1PET4["Average"] = H1PET4.mean( axis=1 )

In [16]:
H0PTPET1 = H0PET1.pivot( index="Year", columns="Month", values="Average" )
H0PTPET2 = H0PET2.pivot( index="Year", columns="Month", values="Average" )
H0PTPET3 = H0PET3.pivot( index="Year", columns="Month", values="Average" )
H0PTPET4 = H0PET4.pivot( index="Year", columns="Month", values="Average" )

In [17]:
H1PTPET1 = H1PET1.pivot( index="Year", columns="Month", values="Average" )
H1PTPET2 = H1PET2.pivot( index="Year", columns="Month", values="Average" )
H1PTPET3 = H1PET3.pivot( index="Year", columns="Month", values="Average" )
H1PTPET4 = H1PET4.pivot( index="Year", columns="Month", values="Average" )

In [18]:
OFNamer = "WSPETMon_%s_PivotTables.xlsx" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
with pd.ExcelWriter( OFPath ) as writer:
    H0PTPET1.to_excel( writer, sheet_name='H0_PET1', na_rep=str(np.nan),
                        index_label="Year" )
    H0PTPET2.to_excel( writer, sheet_name='H0_PET2', na_rep=str(np.nan),
                        index_label="Year" )
    H0PTPET3.to_excel( writer, sheet_name='H0_PET3', na_rep=str(np.nan),
                        index_label="Year" )
    H0PTPET4.to_excel( writer, sheet_name='H0_PET4', na_rep=str(np.nan),
                        index_label="Year" )
    H1PTPET1.to_excel( writer, sheet_name='H1_PET1', na_rep=str(np.nan),
                        index_label="Year" )
    H1PTPET2.to_excel( writer, sheet_name='H1_PET2', na_rep=str(np.nan),
                        index_label="Year" )
    H1PTPET3.to_excel( writer, sheet_name='H1_PET3', na_rep=str(np.nan),
                        index_label="Year" )
    H1PTPET4.to_excel( writer, sheet_name='H1_PET4', na_rep=str(np.nan),
                        index_label="Year" )
# end of with

## Time History Stats

### H0

In [19]:
DictH0PET1 = dict()
DictH0PET2 = dict()
DictH0PET3 = dict()
DictH0PET4 = dict()
hdrCnt = 0
# start out by doing the percentiles
for iI in range(NumQs):
    curQ = OurQs[iI]
    DictH0PET1[StatsHdsList[hdrCnt]] = H0PET1.quantile( q=curQ, axis=1 ).to_numpy()
    DictH0PET2[StatsHdsList[hdrCnt]] = H0PET2.quantile( q=curQ, axis=1 ).to_numpy()
    DictH0PET3[StatsHdsList[hdrCnt]] = H0PET3.quantile( q=curQ, axis=1 ).to_numpy()
    DictH0PET4[StatsHdsList[hdrCnt]] = H0PET4.quantile( q=curQ, axis=1 ).to_numpy()
    hdrCnt += 1
# end for
DictH0PET1[StatsHdsList[hdrCnt]] = H0PET1.mean( axis=1 ).to_numpy()
DictH0PET2[StatsHdsList[hdrCnt]] = H0PET2.mean( axis=1 ).to_numpy()
DictH0PET3[StatsHdsList[hdrCnt]] = H0PET3.mean( axis=1 ).to_numpy()
DictH0PET4[StatsHdsList[hdrCnt]] = H0PET4.mean( axis=1 ).to_numpy()
hdrCnt += 1
DictH0PET1[StatsHdsList[hdrCnt]] = H0PET1.var( axis=1 ).to_numpy()
DictH0PET2[StatsHdsList[hdrCnt]] = H0PET2.var( axis=1 ).to_numpy()
DictH0PET3[StatsHdsList[hdrCnt]] = H0PET3.var( axis=1 ).to_numpy()
DictH0PET4[StatsHdsList[hdrCnt]] = H0PET4.var( axis=1 ).to_numpy()
hdrCnt += 1
DictH0PET1[StatsHdsList[hdrCnt]] = H0PET1.skew( axis=1 ).to_numpy()
DictH0PET2[StatsHdsList[hdrCnt]] = H0PET2.skew( axis=1 ).to_numpy()
DictH0PET3[StatsHdsList[hdrCnt]] = H0PET3.skew( axis=1 ).to_numpy()
DictH0PET4[StatsHdsList[hdrCnt]] = H0PET4.skew( axis=1 ).to_numpy()
hdrCnt += 1
DictH0PET1[StatsHdsList[hdrCnt]] = H0PET1.kurt( axis=1 ).to_numpy()
DictH0PET2[StatsHdsList[hdrCnt]] = H0PET2.kurt( axis=1 ).to_numpy()
DictH0PET3[StatsHdsList[hdrCnt]] = H0PET3.kurt( axis=1 ).to_numpy()
DictH0PET4[StatsHdsList[hdrCnt]] = H0PET4.kurt( axis=1 ).to_numpy()

In [20]:
TimIndex1 = H0PET1.index
TimIndex2 = H0PET2.index
TimIndex3 = H0PET3.index
TimIndex4 = H0PET4.index

In [21]:
H0PETStats1 = pd.DataFrame(index=TimIndex1, data=DictH0PET1 )
H0PETStats2 = pd.DataFrame(index=TimIndex2, data=DictH0PET2 )
H0PETStats3 = pd.DataFrame(index=TimIndex3, data=DictH0PET3 )
H0PETStats4 = pd.DataFrame(index=TimIndex4, data=DictH0PET4 )

### H1

In [22]:
DictH1PET1 = dict()
DictH1PET2 = dict()
DictH1PET3 = dict()
DictH1PET4 = dict()
hdrCnt = 0
# start out by doing the percentiles
for iI in range(NumQs):
    curQ = OurQs[iI]
    DictH1PET1[StatsHdsList[hdrCnt]] = H1PET1.quantile( q=curQ, axis=1 ).to_numpy()
    DictH1PET2[StatsHdsList[hdrCnt]] = H1PET2.quantile( q=curQ, axis=1 ).to_numpy()
    DictH1PET3[StatsHdsList[hdrCnt]] = H1PET3.quantile( q=curQ, axis=1 ).to_numpy()
    DictH1PET4[StatsHdsList[hdrCnt]] = H1PET4.quantile( q=curQ, axis=1 ).to_numpy()
    hdrCnt += 1
# end for
DictH1PET1[StatsHdsList[hdrCnt]] = H1PET1.mean( axis=1 ).to_numpy()
DictH1PET2[StatsHdsList[hdrCnt]] = H1PET2.mean( axis=1 ).to_numpy()
DictH1PET3[StatsHdsList[hdrCnt]] = H1PET3.mean( axis=1 ).to_numpy()
DictH1PET4[StatsHdsList[hdrCnt]] = H1PET4.mean( axis=1 ).to_numpy()
hdrCnt += 1
DictH1PET1[StatsHdsList[hdrCnt]] = H1PET1.var( axis=1 ).to_numpy()
DictH1PET2[StatsHdsList[hdrCnt]] = H1PET2.var( axis=1 ).to_numpy()
DictH1PET3[StatsHdsList[hdrCnt]] = H1PET3.var( axis=1 ).to_numpy()
DictH1PET4[StatsHdsList[hdrCnt]] = H1PET4.var( axis=1 ).to_numpy()
hdrCnt += 1
DictH1PET1[StatsHdsList[hdrCnt]] = H1PET1.skew( axis=1 ).to_numpy()
DictH1PET2[StatsHdsList[hdrCnt]] = H1PET2.skew( axis=1 ).to_numpy()
DictH1PET3[StatsHdsList[hdrCnt]] = H1PET3.skew( axis=1 ).to_numpy()
DictH1PET4[StatsHdsList[hdrCnt]] = H1PET4.skew( axis=1 ).to_numpy()
hdrCnt += 1
DictH1PET1[StatsHdsList[hdrCnt]] = H1PET1.kurt( axis=1 ).to_numpy()
DictH1PET2[StatsHdsList[hdrCnt]] = H1PET2.kurt( axis=1 ).to_numpy()
DictH1PET3[StatsHdsList[hdrCnt]] = H1PET3.kurt( axis=1 ).to_numpy()
DictH1PET4[StatsHdsList[hdrCnt]] = H1PET4.kurt( axis=1 ).to_numpy()

In [23]:
H1PETStats1 = pd.DataFrame(index=TimIndex1, data=DictH1PET1 )
H1PETStats2 = pd.DataFrame(index=TimIndex2, data=DictH1PET2 )
H1PETStats3 = pd.DataFrame(index=TimIndex3, data=DictH1PET3 )
H1PETStats4 = pd.DataFrame(index=TimIndex4, data=DictH1PET4 )

Output to a spreadsheet

In [24]:
OutFileName = "WBMon_PET_%s_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0PETStats1.to_excel( writer, sheet_name="H0PET_1", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H0PETStats2.to_excel( writer, sheet_name="H0PET_2", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H0PETStats3.to_excel( writer, sheet_name="H0PET_3", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H0PETStats4.to_excel( writer, sheet_name="H0PET_4", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H1PETStats1.to_excel( writer, sheet_name="H1PET_1", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H1PETStats2.to_excel( writer, sheet_name="H1PET_2", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H1PETStats3.to_excel( writer, sheet_name="H1PET_3", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
    H1PETStats4.to_excel( writer, sheet_name="H1PET_4", index_label="Date",
                        columns=StatsHdsList, na_rep=str(np.nan) )
# end with